In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/kaggle/input/stocknews/Combined_News_DJIA.csv')

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df[df.isna()['Top25']]

In [ ]:
df.fillna('market remains same', inplace = True)

In [ ]:
df.isna().sum()

In [ ]:
train_df = df[df['Date']<'2014-12-31']
test_df = df[df['Date']>'2014-12-31']

In [ ]:
train_df.shape, test_df.shape

In [ ]:
pip install contractions

In [ ]:
import nltk
import spacy
import re
from bs4 import BeautifulSoup
import requests
import unicodedata
import contractions

nlp = spacy.load('en_core_web_sm')

In [ ]:
def remove_special_characters(text, remove_digits= True):
    #text = text.replace('$', 'currency')
    pattern = r'[^\w]+' if not remove_digits else r'[^a-zA-Z]'
    text = re.sub(pattern," ",text)
    text = re.sub(r'\s+',' ', text)
    return text

def remove_accented_characters(text):
    text =  unicodedata.normalize('NFKD',text).encode('ascii','ignore').decode('utf-8','ignore')
    return text

def spacy_lemma(text):
    text = nlp(text)
    new_text = []
    words = [word.lemma_ for word in text]
    for small in words:
        if small == '-PRON-':
            pass
        else:
            new_text.append(small)

    return ' '.join(new_text)

def contractions_text(text):
    return contractions.fix(text)

def stop_words_removal(text, is_lower_case = False, stopwords = None):
    if stopwords == None:
        stopwords = nlp.Defaults.stop_words
    
    if not is_lower_case:
        text = text.lower()
    tokens = nltk.word_tokenize(text)
    new_token = []
    for i in tokens:
        if len(i)<=1:
            pass
        else:
            new_token.append(i)
    
    removed_text = [word for word in new_token if word not in stopwords]
    
    return ' '.join(removed_text)

def join_news(text):
    full_text = []
    for ind in range(len(text)):
        combine_text = []
        for col in range(2,len(text.columns[2:])+2):
            combine_text.append(text.iloc[ind,col])
        full_text.append(' '.join(combine_text))
    return full_text

In [ ]:
train_data = join_news(train_df)
test_data = join_news(test_df)

In [ ]:
train_data[0]

In [ ]:
test_data[0]

In [ ]:
import tqdm

In [ ]:
def preprocessor_engine(text):
    corpus =[]
    for sent in tqdm.tqdm(text):
        sent = remove_accented_characters(sent)
        sent = contractions_text(sent)
        sent = remove_special_characters(sent)
        sent = spacy_lemma(sent)
        sent = stop_words_removal(sent)
        corpus.append(sent)
    return corpus

In [ ]:
train_data_pro = preprocessor_engine(train_data)

In [ ]:
test_data_pro = preprocessor_engine(test_data)

In [ ]:
train_data_pro[0]

In [ ]:
test_data_pro[0]

#### Tokenizer

In [ ]:
import tensorflow as tf

In [ ]:
tokenzer = tf.keras.preprocessing.text.Tokenizer(oov_token = '<UNK>')
tokenzer.fit_on_texts(train_data_pro)

In [ ]:
#Converting into number token
train_sequences = tokenzer.texts_to_sequences(train_data_pro)
test_sequences = tokenzer.texts_to_sequences(test_data_pro)

In [ ]:
print("Vocabulary size ={}".format(len(tokenzer.word_index)))
print("Number of Documents={}".format(tokenzer.document_count))

In [ ]:
pd.Series(train_data_pro).apply(lambda x : len(x.split())).max()

In [ ]:
MAX_SEQUENCE_LENGTH = 424

train_pad_sequences = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen = MAX_SEQUENCE_LENGTH, padding='post')
test_pad_sequneces = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen = MAX_SEQUENCE_LENGTH, padding='post')

#### Modelling

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, RNN, LSTM, GRU, Bidirectional, Embedding, Dropout
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
y_train = train_df['Label']
y_test = test_df['Label']

In [ ]:
def metrics(y_true, y_pred):
    print('Accuracy Score:', round(accuracy_score(y_true, y_pred),2))
    print('\nClassification Score:\n', classification_report(y_true, y_pred))
    print('\nConfusion Matrix:\n', confusion_matrix(y_true, y_pred))

In [ ]:
def deep_model(layer_name, epochs=50):
    SEED = 42
    np.random.seed(SEED)
    tf.random.set_seed(SEED)

    EMBEDDING_DIM = 300 #Dimension for dense embedding for each token
    VOCAB_SIZE = len(tokenzer.word_index)
    model = Sequential()
    model.add((Embedding(input_dim =VOCAB_SIZE+1,output_dim = EMBEDDING_DIM,input_length = MAX_SEQUENCE_LENGTH)))
    model.add((layer_name(256)))
    model.add((Dense(256,activation = 'relu')))
    model.add(Dense(1,activation = 'sigmoid'))

    model.compile(loss = 'binary_crossentropy',optimizer="adam",metrics =['accuracy'])
    model.summary()
    fit_the_model(model, epochs=50)

In [ ]:
def predictions(model):
    train_pred = model.predict_classes(train_pad_sequences)
    test_pred = model.predict_classes(test_pad_sequneces)
    metrics(y_train, train_pred)
    metrics(y_test, test_pred)

def fit_the_model(model, epochs=50):
    early_stop = EarlyStopping(monitor='val_loss', patience=5)
    model.fit(train_pad_sequences, y_train, epochs=epochs, validation_data=(test_pad_sequneces, y_test), callbacks=[early_stop],verbose=0)
    predictions(model)

In [ ]:
deep_model(LSTM)

In [ ]:
deep_model(GRU)

In [ ]:
def stack_model(layer_name, epochs=50):
    SEED = 42
    np.random.seed(SEED)
    tf.random.set_seed(SEED)

    EMBEDDING_DIM = 300 #Dimension for dense embedding for each token
    VOCAB_SIZE = len(tokenzer.word_index)
    model = Sequential()
    model.add((Embedding(input_dim =VOCAB_SIZE+1,output_dim = EMBEDDING_DIM,input_length = MAX_SEQUENCE_LENGTH)))
    model.add((layer_name(256, return_sequences=True)))
    model.add((layer_name(128, return_sequences=False)))
    model.add((Dense(256,activation = 'relu')))
    model.add(Dense(2,activation = 'softmax'))

    model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(),optimizer="adam",metrics =['accuracy'])
    model.summary()
    fit_the_model(model, epochs=50)

In [ ]:
y_train.value_counts()

In [ ]:
stack_model(LSTM)

In [ ]:
stack_model(GRU)

In [ ]:
def bidirect_model(layer_name, epochs=50, dropout=False):
    SEED = 42
    np.random.seed(SEED)
    tf.random.set_seed(SEED)

    EMBEDDING_DIM = 300 #Dimension for dense embedding for each token
    VOCAB_SIZE = len(tokenzer.word_index)
    model = Sequential()
    model.add((Embedding(input_dim =VOCAB_SIZE+1,output_dim = EMBEDDING_DIM,input_length = MAX_SEQUENCE_LENGTH)))
    
    model.add(Bidirectional(layer_name(256, return_sequences=True)))
    if dropout:
        model.add(Dropout(0.2))
    model.add(Bidirectional(layer_name(128, return_sequences=False)))
    if dropout:
        model.add(Dropout(0.2))
    model.add((Dense(256,activation = 'relu')))
    model.add(Dense(2,activation = 'softmax'))

    model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(),optimizer="adam",metrics =['accuracy'])
    model.summary()
    fit_the_model(model, epochs=50)

In [ ]:
bidirect_model(LSTM)

In [ ]:
#DropOut
bidirect_model(LSTM, dropout=True)

In [ ]:
bidirect_model(GRU)

In [ ]:
#GRU
bidirect_model(GRU, dropout=True)